### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2022
quarter = 1

today = date.today()
today

datetime.date(2022, 5, 4)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2022 AND quarter <= 1) 
OR (year = 2022-1 AND quarter >= 1+1)
ORDER BY year DESC, quarter DESC


In [3]:
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp.shape

(30, 5)

In [4]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(332, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct.sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
17,KSL,2022,Q1,639300,13859,625441,"4,512.89"
6,CIMBT,2022,Q1,3160276,552067,2608209,472.44
5,BH,2022,Q1,1849587,530070,1319517,248.93
3,BBL,2022,Q1,26702007,16433170,10268837,62.49
18,KTB,2022,Q1,24790196,15917212,8872984,55.74


In [6]:
sql = """
DELETE FROM yr_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year, quarter)
rp = conlt.execute(sql)
rp.rowcount

28

In [7]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
rcds = df_ins.values.tolist()
len(rcds)

30

In [8]:
for rcd in rcds:
    print(rcd)

['AOT', 2022, 'Q1', -17151697, -6456020, -10695677, -165.67, 24]
['ASP', 2022, 'Q1', 775000, 741274, 33726, 4.55, 40]
['BAY', 2022, 'Q1', 34707618, 22512053, 12195565, 54.17, 49]
['BBL', 2022, 'Q1', 26702007, 16433170, 10268837, 62.49, 50]
['BFIT', 2022, 'Q1', 696050, 1435683, -739633, -51.52, 58]
['BH', 2022, 'Q1', 1849587, 530070, 1319517, 248.93, 61]
['CIMBT', 2022, 'Q1', 3160276, 552067, 2608209, 472.44, 104]
['COTTO', 2022, 'Q1', 608126, 482435, 125691, 26.05, 710]
['DCC', 2022, 'Q1', 1741605, 1711210, 30395, 1.78, 135]
['DELTA', 2022, 'Q1', 7722365, 8001920, -279555, -3.49, 138]
['DTAC', 2022, 'Q1', 3260225, 4427817, -1167592, -26.37, 144]
['GLOBAL', 2022, 'Q1', 3534362, 2305813, 1228549, 53.28, 193]
['GVREIT', 2022, 'Q1', 672205, 783558, -111353, -14.21, 654]
['HMPRO', 2022, 'Q1', 5588994, 5250650, 338344, 6.44, 208]
['IRC', 2022, 'Q1', 245994, 280507, -34513, -12.3, 225]
['KBANK', 2022, 'Q1', 38636812, 32739104, 5897708, 18.01, 245]
['KKP', 2022, 'Q1', 6910731, 5101876, 1808855

In [9]:
sql = """INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)

INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [10]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [11]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [12]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
17,KSL,2022,Q1,639300,13859,625441,"4,512.89"
6,CIMBT,2022,Q1,3160276,552067,2608209,472.44
5,BH,2022,Q1,1849587,530070,1319517,248.93
3,BBL,2022,Q1,26702007,16433170,10268837,62.49
18,KTB,2022,Q1,24790196,15917212,8872984,55.74


In [13]:
criteria_2 = df_ins.q_amt_p > 300000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
6,CIMBT,2022,Q1,3160276,552067,2608209,472.44
5,BH,2022,Q1,1849587,530070,1319517,248.93
3,BBL,2022,Q1,26702007,16433170,10268837,62.49
18,KTB,2022,Q1,24790196,15917212,8872984,55.74
2,BAY,2022,Q1,34707618,22512053,12195565,54.17


In [14]:
criteria_3 = df_ins.Pct > 5.0
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
17,KSL,2022,Q1,639300,13859,625441,"4,512.89"
6,CIMBT,2022,Q1,3160276,552067,2608209,472.44
5,BH,2022,Q1,1849587,530070,1319517,248.93
3,BBL,2022,Q1,26702007,16433170,10268837,62.49
18,KTB,2022,Q1,24790196,15917212,8872984,55.74


In [15]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
6,CIMBT,2022,Q1,3160276,552067,2608209,472.44
5,BH,2022,Q1,1849587,530070,1319517,248.93
3,BBL,2022,Q1,26702007,16433170,10268837,62.49
18,KTB,2022,Q1,24790196,15917212,8872984,55.74
2,BAY,2022,Q1,34707618,22512053,12195565,54.17
11,GLOBAL,2022,Q1,3534362,2305813,1228549,53.28
23,PTTEP,2022,Q1,37848936,25585211,12263725,47.93
16,KKP,2022,Q1,6910731,5101876,1808855,35.45
24,SCB,2022,Q1,35703880,28054402,7649478,27.27
7,COTTO,2022,Q1,608126,482435,125691,26.05


In [16]:
df_ins.loc[df_ins_criteria,cols].sort_values(['name'],ascending=[True])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
2,BAY,2022,Q1,34707618,22512053,12195565,54.17
3,BBL,2022,Q1,26702007,16433170,10268837,62.49
5,BH,2022,Q1,1849587,530070,1319517,248.93
6,CIMBT,2022,Q1,3160276,552067,2608209,472.44
7,COTTO,2022,Q1,608126,482435,125691,26.05
11,GLOBAL,2022,Q1,3534362,2305813,1228549,53.28
13,HMPRO,2022,Q1,5588994,5250650,338344,6.44
15,KBANK,2022,Q1,38636812,32739104,5897708,18.01
16,KKP,2022,Q1,6910731,5101876,1808855,35.45
18,KTB,2022,Q1,24790196,15917212,8872984,55.74
